# Caso de estudio - Computrabajo Antioquia

En el siguiente estudio se desea mostrar como extraer información de una pagina web con librerias de python y almacenarla en un DataFrame .

**NOTA:** Esta información se recolecto solo para uso ***didactico***,  es su responsabilidad el manejo externo que usted le dara .

# Importamos librerias 

In [1]:
import pandas as pd
import requests
from bs4  import BeautifulSoup

# Extraccción pagina 1

In [2]:
r1=requests.get('https://www.computrabajo.com.co/empleos-en-antioquia')
soup=BeautifulSoup(r1.text,'html.parser')

In [3]:
# Nombre de la vacante 

Job_title=soup.findAll(class_='js-o-link')
Job_title[0]['title'] # Nombre de la vacante 

'Empleo de Convocatoria laboral técnico en instalaciones de redes'

In [4]:
# Nombre de la empresa page 1 

Nombre_empresa1 = soup.find_all(class_='empr it-blank')

Nombre_empresa1[0]['title'].split('Valoraciones de la empresa')[-1] # Nombre de la empresa

' HQ5 S.A.S'

In [5]:
# Descripción de la vacante


Descripcion_vacante=soup.find_all('p')[3:]

Descripcion_vacante[0].text[0:-1] # Descripcion de las vacantes 

'Si eres bachiller, técnico o tecnologo con experiencia RECIENTE en Instalación de Internet telefonía y parabólica..'

In [6]:
# Fecha de publicación 

Fecha_pub = soup.find_all('span', attrs={'class':'dO'})
Fecha_pub[4].text # Fecha de publicación

'Hoy, 10:53 a. m.'

# DataFrames Vacantes pagina 1

In [7]:
Vacantes1 = []  
for i in range(20):  
    Titulo=Job_title[i]['title']
    Fecha=Fecha_pub[i].text
    Desripcion=Descripcion_vacante[i].text[0:-1]
    Empresa=Nombre_empresa1[i]['title'].split('Valoraciones de la empresa')[-1]
    Vacantes1.append((Fecha,Titulo,Desripcion,Empresa))

In [8]:
df1 = pd.DataFrame(Vacantes1, columns=['Fecha','Titulo','Desripcion','Empresa'])

In [9]:
df1.head()

,Fecha,Titulo,Desripcion,Empresa
0,"Hoy, 11:06 a. m.",Empleo de Convocatoria laboral técnico en inst...,"Si eres bachiller, técnico o tecnologo con exp...",HQ5 S.A.S
1,"Hoy, 11:01 a. m.",Empleo de Coordinador Comercial Apartado,Empresa de telecomunicaciones requiere Coordin...,SU TEMPORAL S.A
2,"Hoy, 11:01 a. m.",Empleo de Mensajero - Motorizado,Reconocida compañía Importadora requiere Mensa...,
3,"Hoy, 10:56 a. m.","Empleo de Asesor de Repuestos, Filtros y Lubri...",Se necesita persona con PERFIL COMERCIAL y exp...,TECNODIESEL S.A.S
4,"Hoy, 10:53 a. m.",Empleo de conductor con licencia c2 o c3 urgen...,importante empresa de consumo masivo requiere ...,PROCESOS TERCERIZADOS S.A.S


# Extraccion paginas restantes desde la 2 a la 715

En esta sección vamos a hacer uso de la tecnica **`Crawling horizontal`** para poder tener acceso a la información de todas las paginas restantes. 

In [10]:
url=[]  # Vector donde se cargaran la información de todas las url's
for i in range(2,715):
    url.append('https://www.computrabajo.com.co/empleos-en-antioquia?p=' + str(i))

In [11]:
Vacantes2=[]
for i in range(712):
    pages=requests.get(url[i])
    soup=BeautifulSoup(pages.text,'html.parser')
    Job_title=soup.findAll(class_='js-o-link')
    Nombre_empresa1 = soup.find_all(class_='empr it-blank')
    Descripcion_vacante=soup.find_all('p')[3:]
    Fecha_pub = soup.find_all('span', attrs={'class':'dO'})
    
    # Ciclo de extraccion por pagina 
    
    for i in range(20):  
        Titulo=Job_title[i]['title']
        Fecha=Fecha_pub[i].text
        Desripcion=Descripcion_vacante[i].text[0:-1]
        Empresa=Nombre_empresa1[i]['title'].split('Valoraciones de la empresa')[-1]
        Vacantes2.append((Fecha,Titulo,Desripcion,Empresa)) 

In [12]:
df2 = pd.DataFrame(Vacantes2, columns=['Fecha','Titulo','Desripcion','Empresa'])

# Concatenamos los 2 DataFrames

In [13]:
Vacantes_final=pd.concat([df1,df2],ignore_index=True)

In [17]:
Vacantes_final.info() # Se extrajeron 14260 vacantes de Antioquia

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14260 entries, 0 to 14259
Data columns (total 4 columns):
Fecha         14260 non-null object
Titulo        14260 non-null object
Desripcion    14260 non-null object
Empresa       14260 non-null object
dtypes: object(4)
memory usage: 445.7+ KB


In [18]:
Vacantes_final.head()

,Fecha,Titulo,Desripcion,Empresa
0,"Hoy, 11:06 a. m.",Empleo de Convocatoria laboral técnico en inst...,"Si eres bachiller, técnico o tecnologo con exp...",HQ5 S.A.S
1,"Hoy, 11:01 a. m.",Empleo de Coordinador Comercial Apartado,Empresa de telecomunicaciones requiere Coordin...,SU TEMPORAL S.A
2,"Hoy, 11:01 a. m.",Empleo de Mensajero - Motorizado,Reconocida compañía Importadora requiere Mensa...,
3,"Hoy, 10:56 a. m.","Empleo de Asesor de Repuestos, Filtros y Lubri...",Se necesita persona con PERFIL COMERCIAL y exp...,TECNODIESEL S.A.S
4,"Hoy, 10:53 a. m.",Empleo de conductor con licencia c2 o c3 urgen...,importante empresa de consumo masivo requiere ...,PROCESOS TERCERIZADOS S.A.S


# Creación del archivo final csv

In [16]:
Vacantes_final.to_csv('Vacantes_Antioquia.csv')

# FINAL...........